In [3]:
import mitsuba as mi
import numpy as np
mi.set_variant('cuda_ad_rgb')

[mitsuba] Warning: Couldn't import the ipywidgets package. Installing this package is required for the system to properly log messages and print in Jupyter notebooks!


In [4]:
import json

import numpy as np
import drjit as dr

class MitsubaJSONEncoder(json.JSONEncoder):
    def default(self, obj):
        # Handle ScalarVector2u and other drjit vector types
        if hasattr(obj, '__class__') and ('ScalarVector' in obj.__class__.__name__ or 
                                        'Vector' in obj.__class__.__name__):
            return list(obj)
        
        # Handle Transform4f
        if hasattr(obj, '__class__') and 'Transform' in obj.__class__.__name__:
            try:
                # Extract the matrix data from the transform
                matrix = obj.matrix
                return matrix.numpy().tolist()
            except:
                # Fallback: try to convert to list directly
                return list(obj)
        
        # Handle drjit arrays
        if dr.is_array_v(obj):
            try:
                return dr.detach(obj).numpy().tolist()
            except:
                return list(obj)
        
        # Handle numpy arrays and matrices
        if isinstance(obj, np.ndarray):
            return obj.tolist()
            
        # Handle other array-like objects
        try:
            if hasattr(obj, '__iter__') and not isinstance(obj, (str, dict)):
                return list(obj)
        except:
            pass
            
        # Handle nested structures
        if isinstance(obj, (list, tuple)):
            return [self.default(item) for item in obj]
            
        # For all other types, try to convert to a basic Python type
        try:
            # Handle float types
            if hasattr(obj, 'item'):
                return obj.item()
            # Handle other numeric types
            elif hasattr(obj, '__float__'):
                return float(obj)
            elif hasattr(obj, '__int__'):
                return int(obj)
        except:
            pass
            
        return super().default(obj)

def serialize_mitsuba_scene(params_dict):
    """
    Serialize Mitsuba scene parameters to JSON with proper type handling.
    
    Args:
        params_dict (dict): Dictionary containing Mitsuba scene parameters
        
    Returns:
        str: JSON string representation of the scene parameters
    
    Example:
        scene = mi.load_file('scene.xml')
        params = mi.traverse(scene)
        params_dict = {a[0]: a[1] for a in params}
        json_output = serialize_mitsuba_scene(params_dict)
    """
    return json.dumps(params_dict, cls=MitsubaJSONEncoder, indent=2)

def deserialize_mitsuba_scene(json_str):
    """
    Deserialize JSON string back into a Python dictionary.
    Note: This converts the data back into basic Python types,
    not into Mitsuba-specific types.
    
    Args:
        json_str (str): JSON string of scene parameters
        
    Returns:
        dict: Dictionary containing the deserialized scene parameters
    """
    return json.loads(json_str)

## Change the Scene parameters

In [6]:
import json

scene = mi.load_file('../scenes/true_scene.xml')

# Traverse the scene to find the parameters
params = mi.traverse(scene)
params_dict = {}
for i, a in enumerate(params):
    params_dict[a[0]] = a[1]

# Use the custom serializer
json_output = serialize_mitsuba_scene(params_dict)
print(json_output)

{
  "PerspectiveCamera.near_clip": 0.009999999776482582,
  "PerspectiveCamera.far_clip": 10000.0,
  "PerspectiveCamera.shutter_open": 0.0,
  "PerspectiveCamera.shutter_open_time": 0.0,
  "PerspectiveCamera.film.size": [
    512,
    512
  ],
  "PerspectiveCamera.film.crop_size": [
    512,
    512
  ],
  "PerspectiveCamera.film.crop_offset": [
    0,
    0
  ],
  "PerspectiveCamera.x_fov": [
    20.0
  ],
  "PerspectiveCamera.principal_point_offset_x": [
    0.0
  ],
  "PerspectiveCamera.principal_point_offset_y": [
    0.0
  ],
  "PerspectiveCamera.to_world": [
    [
      [
        -1.0,
        0.0,
        0.0,
        0.0
      ],
      [
        0.0,
        1.0,
        0.0,
        0.0
      ],
      [
        0.0,
        0.0,
        -1.0,
        0.05000000074505806
      ],
      [
        0.0,
        0.0,
        0.0,
        1.0
      ]
    ]
  ],
  "BlackWallBSDF.brdf_0.reflectance.value": [
    [
      0.05999999865889549,
      0.05999999865889549,
      0.05999999865

In [15]:
param_keys = ['Cube.interior_medium.albedo.value.value', 'Cube.interior_medium.sigma_t.value.value', 'Cube.interior_medium.phase_function.g']

true_param_values = {}
for key in param_keys:
    if key in params:
        true_params = params[key]
        try:
            true_param_values[key] = np.array(true_params).flatten()
        except:
            raise TypeError(f"Unsupported type for parameter '{key}': {type(true_params)}")

In [16]:
print(true_param_values)

{'Cube.interior_medium.albedo.value.value': array([0.99, 0.99, 0.99], dtype=float32), 'Cube.interior_medium.sigma_t.value.value': array([0.2, 0.5, 0.9], dtype=float32), 'Cube.interior_medium.phase_function.g': array([0.1], dtype=float32)}


In [5]:
params['PerspectiveCamera.film.size'][0]

256

In [20]:
# Render the scene and save the result to a file
image = mi.render(scene)
mi.util.write_bitmap('scene_before.png', image, write_async=True)

In [21]:
# Modify the sigma_t parameter of the interior medium of the Cube
params['Cube.interior_medium.sigma_t.value.value'] = [0.2, 0.2, 0.2]

In [22]:
# Render the scene with the modified parameter and save the result to a file
image = mi.render(scene)
mi.util.write_bitmap('scene_after.png', image, write_async=True)

## Check inside the image

In [2]:
scene = mi.load_file('scene.xml')

In [3]:
image = mi.render(scene)
np_image = np.array(image)

In [4]:
# Check the maximum value for each channel
np_image.max(axis=(0, 1))

array([0.165036, 0.165036, 0.165036], dtype=float32)

In [5]:
# Traverse the scene to find the parameters
params = mi.traverse(scene)

params['SpotLight.intensity.value']

[[1.0, 1.0, 1.0]]

In [6]:
# Change the intensity of the SpotLight
params['SpotLight.intensity.value'] = [100, 100, 100]

In [7]:
image = mi.render(scene)

In [ ]:
# save the result to a bitmap file
mi.util.write_bitmap('scene.png', image, write_async=True)

In [8]:
# Check the maximum value for each channel
np_image = np.array(image)
np_image.max(axis=(0, 1))

array([16.503595, 16.503595, 16.503595], dtype=float32)

In [10]:
# save to csv
np.savetxt('akarui.csv', np_image.reshape(-1, 3), delimiter=',', fmt='%f')